In [89]:
import pandas as pd
import os

In [18]:
def extractProteinLength(file):
    f = open(file)
    f.readline()
    lengths=dict()
    for line in f:
        entry, organism, ec, sequence = line.strip().split("\t")
        lengths[entry]=len(sequence)
    return lengths

In [21]:
FILE_CYTOPLASMIC="../2020-04-06.FindTwinsAnyEvidenceBacteria/1.downloadProteins/proteinsCytoplasmic.tab"
FILE_PERIPLASMIC="../2020-04-06.FindTwinsAnyEvidenceBacteria/1.downloadProteins/proteinsPeriplasmic.tab"

In [22]:
LENGTHS_CYTOPLASMIC = extractProteinLength(FILE_CYTOPLASMIC)
LENGTHS_PERIPLASMIC = extractProteinLength(FILE_PERIPLASMIC)

In [85]:
def ratioWithoutSP(row):
    CytoLength=row["Cytoplasm_length"]
    PeriLength=row["Periplasm_length"]-row["SP_end"]+row["SP_start"]
    Max = max(CytoLength,PeriLength)
    Min = min(CytoLength,PeriLength)
    return Min/Max

def absDifference(row):
    CytoLength=row["Cytoplasm_length"]
    PeriLength=row["Periplasm_length"]-row["SP_end"]+row["SP_start"]
    Max = max(CytoLength,PeriLength)
    Min = min(CytoLength,PeriLength)
    return Max-Min
    


# Read in Twins
TWINS=pd.read_csv("twins_with_org_sp.tab", sep="\t")
TWINS["Cytoplasm_length"]=TWINS["Cytoplasm"].apply(lambda x:LENGTHS_CYTOPLASMIC[x])
TWINS["Periplasm_length"]=TWINS["Periplasm"].apply(lambda x:LENGTHS_PERIPLASMIC[x])
TWINS["ratio_length"]=TWINS.apply(ratioWithoutSP,axis=1)
TWINS["diff_length"]=TWINS.apply(absDifference, axis=1)
TWINS_WITH_LENGTH=TWINS.sort_values(by="diff_length")
TWINS_WITH_LENGTH

,Cytoplasm,Periplasm,Organism,SP_start,SP_end,Cytoplasm_length,Periplasm_length,ratio_length,diff_length
1252,A0A169XQ31,A0A384I9X6,Klebsiella_oxytoca,1,34,549,582,1.000000,0
1225,A0A169XQ31,A0A0G3S663,Klebsiella_oxytoca,1,34,549,582,1.000000,0
1243,A0A169XQ31,A0A3E1SVQ6,Klebsiella_oxytoca,1,34,549,582,1.000000,0
1261,A0A169XQ31,A0A2I8WC74,Klebsiella_oxytoca,1,34,549,582,1.000000,0
12092,A0A379VJE2,A0A3Y2F011,Salmonella_enterica_I,1,34,541,574,1.000000,0
...,...,...,...,...,...,...,...,...,...
7616,A0A2V9DP80,A0A3N5ZMI4,Acidobacteria_bacterium,1,19,1161,349,0.285099,830
7622,A0A2V9QGT4,A0A3N5ZMI4,Acidobacteria_bacterium,1,19,1177,349,0.281223,846
7591,A0A522PXJ5,A0A3N5ZMI4,Acidobacteria_bacterium,1,19,1179,349,0.280746,848
7592,A0A522PXJ5,A0A3N5ZMI4,Acidobacteria_bacterium,1,19,1179,349,0.280746,848


In [88]:
TWINS_WITH_LENGTH.to_csv("twins_with_length_diff.tab",sep="\t")

In [102]:
DIR="../2020-04-06.FindTwinsAnyEvidenceBacteria/4.7.clustalOmega/"
PROT_WITH_MSA = pd.DataFrame([F.split(".")[0] for F in os.listdir(DIR) if F.endswith(".fasta")], columns=["Cytoplasm"])
PROT_WITH_MSA["Periplasm"]=PROT_WITH_MSA["Cytoplasm"]
PROT_WITH_MSA

,Cytoplasm,Periplasm
0,A0A3E1KBE5,A0A3E1KBE5
1,A0A497DY78,A0A497DY78
2,A0A5B8YGN9,A0A5B8YGN9
3,A0A4S3K8T0,A0A4S3K8T0
4,A0A523KDX2,A0A523KDX2
...,...,...
381,A0A1F9S2E0,A0A1F9S2E0
382,A0A0S7XDF4,A0A0S7XDF4
383,A0A2E9N0E5,A0A2E9N0E5
384,P0AAL1,P0AAL1


In [116]:
EXAMPLES = TWINS_WITH_LENGTH.merge(PROT_WITH_MSA["Cytoplasm"],on="Cytoplasm").merge(PROT_WITH_MSA["Periplasm"],on="Periplasm").sort_values("diff_length")
EXAMPLES.groupby(["Cytoplasm","Periplasm"])["diff_length"].min().sort_values().head(20)

Cytoplasm   Periplasm 
Q9ZGM4      Q9WXB9         2
A0A486SVC6  A0A377XXY4     2
P23869      P0AFL3         3
P12994      P77368         5
A0A379VJE2  A0A3T4VNE6     9
A0A0M0QC81  A0A3X7IDQ5    10
P0A962      P00805        11
A0A5I0XCE4  A0A5I5D911    12
A0A5I2PB29  A0A5I2P717    12
A0A5I2WYV8  A0A5I2WX39    12
A0A5J0NH77  A0A5I5EW62    12
Q72CB8      Q72EC8        12
A0A5I3D2A2  A0A5I3D1C4    12
A0A5F1DXY5  A0A5F1E5W7    13
A0A376LD21  A0A0K4HJS0    15
A0A3U3D778  A0A3T4VNE6    17
P0AAL1      P0AAL4        27
A0A0H2V0P7  P0AAL5        27
A0A193QGU2  Q2NVU4        28
Q07YY9      Q9Z4P0        29
Name: diff_length, dtype: int64